<a href="https://colab.research.google.com/github/Divyam-Deep/Guess-the-word-/blob/main/Guess_the_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import gensim.downloader as api
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import random
import time

In [20]:
# Function to load Word2Vec model with retries
def load_word2vec_model(model_name, retries=3, wait_time=5):
    for attempt in range(retries):
        try:
            wv = api.load(model_name)
            return wv
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
    # Fallback to a smaller model if retries are exhausted
    print("Falling back to a smaller model: glove-wiki-gigaword-50")
    return api.load('glove-wiki-gigaword-50')


In [21]:
# Load pre-trained Word2Vec model
wv = load_word2vec_model('word2vec-google-news-300')


In [22]:
# Predefined list of common words (for example, this can be expanded or refined)
common_words = [
    'time', 'person', 'year', 'way', 'day', 'thing', 'man', 'world', 'life', 'hand',
    'part', 'child', 'eye', 'woman', 'place', 'work', 'week', 'case', 'point', 'government',
    'company', 'number', 'group', 'problem', 'fact', 'apple', 'banana', 'orange', 'grape',
    'breakfast', 'morning', 'evening', 'computer', 'science', 'music', 'school', 'teacher',
    'student', 'friend', 'family', 'mother', 'father', 'brother', 'sister', 'house', 'car',
    'city', 'country', 'water', 'food', 'dog', 'cat', 'mouse', 'bird', 'fish', 'tree', 'flower'
]

In [23]:
# Filter words to ensure they are in the Word2Vec model
filtered_common_words = [word for word in common_words if word in wv]


In [24]:
# Function to generate a random word
def generate_random_word():
    return np.random.choice(filtered_common_words)


In [25]:
# Function to compute similarity
def compute_similarity(word1, word2):
    if word1 in wv and word2 in wv:
        vec1 = wv[word1].reshape(1, -1)
        vec2 = wv[word2].reshape(1, -1)
        similarity = cosine_similarity(vec1, vec2)
        return similarity[0][0]
    else:
        return 0.0

In [26]:
# Function to find similar words, synonyms, or antonyms
def find_similar_words(word, topn=10):
    if word in wv:
        similar_words = wv.most_similar(word, topn=topn)
        # Filter similar words to keep only those in the filtered_common_words list
        filtered_similar_words = [sim_word for sim_word, _ in similar_words if sim_word in filtered_common_words]

        suggested_words = []

        # Collect up to 2 most similar words, synonyms, or antonyms
        for sim_word in filtered_similar_words:
            if sim_word != word:
                suggested_words.append(sim_word)
            if len(suggested_words) >= 2:
                break

        # If fewer than 2 words are found, add random common words
        while len(suggested_words) < 2:
            random_word = generate_random_word()
            if random_word not in suggested_words and random_word != word:
                suggested_words.append(random_word)

        return suggested_words[:2]  # Return up to 2 suggested words
    else:
        # If word is not in the Word2Vec model, return an empty list
        return []


In [27]:
# Function to create a hint sentence
def create_hint_sentence(similar_words):
    hints = ", ".join(similar_words[:2])
    return f"A word similar to this: {hints}"

In [28]:
# Function to display similarity as a bar
def display_similarity(attempts, similarities):
    print("\nGuesses and their similarities:")
    for attempt, sim in zip(attempts, similarities):
        bar = '█' * int(sim * 50)
        print(f"{attempt}: {sim:.2f} {bar}")

In [29]:
# def run_game_timer():
#     start_time = time.time()
#     while time.time() - start_time < 60:
#         remaining_time = int(60 - (time.time() - start_time))
#         clear_output(wait=True)
#         print(f"Time left: {remaining_time} seconds")
#         user_input = input("\nEnter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ").strip().lower()
#         if user_input in ["new", "n", "ans", "answer"]:
#             return user_input, True
#     return None, False

In [30]:
# Game logic
def word_guessing_game():
    score = 0
    while True:
        generated_word = generate_random_word()
        similar_words = find_similar_words(generated_word)
        hint_sentence = create_hint_sentence(similar_words)
        print('Rules: \nYou Have Three Attempts to guess the word\ntype new to start a new game \nans to reveal the word')
        print(f"\nGame started. \nHint: The word has {len(generated_word)} letters.")
        print(hint_sentence)

        attempts = []
        similarities = []

        attempts_left = 3 # Number of attempts allowed
        while attempts_left > 0:
            user_input = input("\nEnter your guess: ").strip().lower()
            if user_input in ["new", "n"]:
                break
            elif user_input in ["ans", "answer"]:
                break
            elif user_input == generated_word:
                print("Congratulations! You've guessed the correct word!")
                score += 10  # Increase score for correct guess
                break

            similarity = compute_similarity(user_input, generated_word)
            attempts.append(user_input)
            similarities.append(similarity)
            attempts_left -= 1

            clear_output(wait=True)
            print(f"Hint: The word has {len(generated_word)} letters.")
            print(hint_sentence)
            display_similarity(attempts, similarities)
            if attempts_left > 0:
               print(f"Attempts Left: {(attempts_left)}")
            else:
               continue


        if user_input == generated_word:
            print(f"Your Current Score: {score}")

        elif user_input in ["new", "n"]:
            print(f"Sorry, you did not guess the word. The correct word was: {generated_word}")
            print(f"Your Final Score: {score}")
            play_again = input("\nDo you want to play again? (yes/no): ").strip().lower()
            if play_again != "yes":
                break  # Exit the game loop
            score = 0  # Reset score for a new game
            clear_output(wait=True)
        else:
            print(f"Sorry, you did not guess the word. The correct word was: {generated_word}")
            print(f"Your Final Score: {score}")
            play_again = input("\nDo you want to play again? (yes/no): ").strip().lower()
            if play_again != "yes":
                break  # Exit the game loop
            score = 0  # Reset score for a new game
            clear_output(wait=True)


In [31]:
# Run the game
word_guessing_game()

Hint: The word has 6 letters.
A word similar to this: person, life

Guesses and their similarities:
alive: 0.04 ██
Attempts Left: 2

Enter your guess: ans
Sorry, you did not guess the word. The correct word was: orange
Your Final Score: 0

Do you want to play again? (yes/no): no
